# Tutorial 6 - Python for Data Analysis
---
## Selenium

  - [Part 1](#exercise-1) : Installation
  - [Part 2](#exercise-2) : LinkedIn Login & Profile
  - [Part 3](#exercise-3) : LinkedIn Relations
  - [Bonus](#exercise-bonus) : Portail


For Selenium to be able to work, we need to download the adequate "webdriver" based on your browser.
I will be giving you the steps for Chrome, however they are sensibly the same for any web browser.

First step : Check your browser version. For Chrome, you need to visit "help" -> "about Chrome".

Second step : Download webdriver for the version of your browser here :
https://googlechromelabs.github.io/chrome-for-testing/ (you should have V.118 of Chrome)

Third step : choose the link with binary "webdriver" and your OS (mac, windows) to download the webdriver, then place it ideally in your project directory.

Fourth step : Either set your webdriver to be a PATH executable OR ignore this step and use the argument executable_path in the webdriver object definition

<a name="exercise-1">

### Part 1 : Installation
---

In [2]:
!pip install -U selenium


[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install lxml

     ---------------------------------------- 3.8/3.8 MB 3.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip list

Package                   Version
------------------------- ------------
anyio                     4.0.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.0
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.0
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.1.0
certifi                   2023.7.22
cffi                      1.16.0
charset-normalizer        3.3.0
colorama                  0.4.6
comm                      0.1.4
contourpy                 1.1.1
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.0
fastjsonschema            2.18.1
fonttools                 4.43.1
fqdn                      1.5.1
h11                       0.14.0
idna                      3.4
ipykernel                 6.25.2
ipython           


[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install beautifulsoup4


[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Create your first driver !

Install the browser driver depending on your browser version : https://www.selenium.dev/documentation/webdriver/getting_started/install_drivers/

Place the exe file in the directory of your jupyterlab directory.
For the latest documentation on selenium : https://www.selenium.dev/documentation/overview/

In [8]:
driver = webdriver.Chrome()
driver.get("https://www.google.com")

In [ ]:
#driver.quit()

<a name="exercise-2">

### Part 2 : LinkedIn Login
---

1) Find the elements corresponding to the login field and fill them by inspecting the linkedin page

In [ ]:
import getpass

2) Find two ways of login in : with submit button and keys.

3) Find the button to go on your profile page and automate the action of clicking.

4) Use a script to scroll the page to the bottom to browse all objects (execute_script function)

5) Use beautifulsoup and process the HTML of the previous scrolled page

In [ ]:
from bs4 import BeautifulSoup

6.1) Extract under the variable name "section" the HTML part of the page corresponding to your experiences using the section ID

6.2) As you can see, if you refresh the page the section ID changes, what can you do to always find it back ? Create a function to solve this question.

In [ ]:
import numpy as np

7) From this extract part, extract separately experiences and from experiences : companies, period of time, description... using find_all (you might have doubled data but we will filter it after)

8) Create a format_data function to process every list and keep only one title inside of a list

9) Create a dataframe named my_linkedin_data and insert all that data inside of it.

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

<a name="exercise-3">

### Part 3 : LinkedIn Relations
---

1) Click on the "More relations" button to open your list of relations

2) Extract the names and the jobs of your relations on the page.

3) Find a way to navigate automatically between pages and extract all names, jobs, locationsand store them in a Dataframe

In [ ]:
import time